## Interactive (html) maps

This notebook contains a set of examples on how to create simple html maps

In [1]:
# Import folium package
import folium

### Example #1: Basic map

In [2]:
# Create base map
m = folium.Map( location=[37.86, -122.26], 
                tiles="OpenStreetMap",
                 zoom_start = 12)

# Visualize map on jupyter notebook
m


Try different backgrounds (i.e., tiles)

In [3]:
folium.TileLayer('OpenStreetMap').add_to(m)
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('Stamen Toner').add_to(m)
folium.TileLayer('Stamen Water Color').add_to(m)
folium.TileLayer('cartodbpositron').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)

folium.LayerControl().add_to(m)

m

Add a cicle

In [4]:
folium.Circle( radius=100, # in pixels
               location=[37.87463783421649, -122.25949101944815],
               color='crimson',
               fill=False).add_to(m)

m

Add a simple marker

In [5]:
# "Normal" marker
folium.Marker(
    [37.87463783421649, -122.25949101944815], popup="<i>Office</i>",
    tooltip = "Click me!" ).add_to(m)
m

Fancier markers

In [6]:
folium.Marker(location=[37.87563167364989, -122.26041017369546],
              popup='<h5 style="color:green;"> You found coffee! </h5>',
              tooltip='<strong>Click here to see Popup</strong>').add_to(m)

m

Change Icon

In [7]:
folium.Marker(location=[37.87563167364989, -122.26041017369546],
              popup='<h5 style="color:green;"> Coffee </h5>',
              icon=folium.Icon(color='green', icon='info-sign'),
              tooltip='<strong>Click here to see Popup</strong>').add_to(m)

m



Circular marker

In [8]:
folium.CircleMarker(
    location=[37.87106052510632, -122.25107632745537],
    radius=10,
    popup="Stadium",
    color="#3186cc",
    fill=True,
    fill_color="#3186cc",
).add_to(m)

m

Save map

In [9]:
m.save("output/example_1.html")

## Example 2

Visualize waveforms from an event: [Mww7.6 South Of Alaska earthquake](https://ds.iris.edu/wilber3/find_stations/11327190)

Original obspy example:
https://www.earthinversion.com/geophysics/getting-started-with-obspy-for-seismologists-part-I/

In [10]:
# Import additional packages
from obspy import read
from obspy.clients.fdsn import Client
from obspy import UTCDateTime, read_inventory
import matplotlib.pyplot as plt


In [11]:
# Initialize client
client = Client( "IRIS" )  

# uses Obspy definition of datetime 
eventTime = UTCDateTime("2020-10-19T20:54:39")
starttime = eventTime - 60     # 1 minute before the event
endtime = eventTime + 15 * 60  # 15 minutes after the event

net = "IU"  # network of the station
sta = "*"   # station code
loc = "00"  # to specify the instrument at the station
chan = "BHZ"

# Read inventory
inv = read_inventory()

# Download waveform data and store it into a stream object
myStream = client.get_waveforms(
    net, sta, loc, chan, starttime, endtime, attach_response=True )

# Get station information
inventory = client.get_stations( network = net,
                                 station = sta,
                                 starttime = starttime,
                                 channel = chan,
                                 location = loc,
                                 endtime = endtime )

# Remove instrument reponse
st_rem = myStream.copy()
st_rem.remove_response(output="VEL")  # DISP, ACC

# Plot the data and save plots as png files
for trace in st_rem:
    trace.plot(
        outfile= "output/"+trace.stats.station+".png",
        starttime=None,
        endtime=None,
        size=(800, 250),
        dpi=100,
        color="blue",
        bgcolor="white",
        face_color="white",
        transparent=False,
        number_of_ticks=6,
        tick_rotation=45,
        type="relative",
        linewidth=0.5,
        linestyle="-" )


In [12]:
# Create base map
m = folium.Map( location=[0, 0], 
                tiles="OpenStreetMap",
                 zoom_start = 1)

# Add stations to map
for station in inventory[0]:
    # Create html code to link to figure
    html_code = '\n <p> <img src=\"'+station.code+'.png\" style="width:800px;height:300px;">\n </p>\n'
        
    folium.Marker(location = [station.latitude, station.longitude],
                  popup = html_code,
                  tooltip = station.code).add_to(m)
    
# Plot event
elat = 54.6093
elon = -159.6519

folium.CircleMarker(
    location=[elat, elon],
    radius=10,
    popup="Event",
    color="red",
    fill=True,
    fill_color="red",
).add_to(m)


# Visualize map on jupyter notebook
m.save("output/example_2.html" )

## Example 3

Add plots using json objects

In [13]:
# Import additional packages
import json
import requests

In [14]:
# Download json data
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
vis1 = json.loads(requests.get(f"{url}/vis1.json").text)
vis2 = json.loads(requests.get(f"{url}/vis2.json").text)
vis3 = json.loads(requests.get(f"{url}/vis3.json").text)

# Create a new basemap

m = folium.Map(location=[46.3014, -123.7390], zoom_start=7, tiles="Stamen Terrain")

folium.Marker(
    location=[47.3489, -124.708],
    popup=folium.Popup(max_width=450).add_child(
        folium.Vega(vis1, width=450, height=250)
    ),
).add_to(m)

folium.Marker(
    location=[44.639, -124.5339],
    popup=folium.Popup(max_width=450).add_child(
        folium.Vega(vis2, width=450, height=250)
    ),
).add_to(m)

folium.Marker(
    location=[46.216, -124.1280],
    popup=folium.Popup(max_width=450).add_child(
        folium.Vega(vis3, width=450, height=250)
    ),
).add_to(m)


m



## Example 4

Heat map - Bike stations in Chicago

Example taken from:
https://livecodestream.dev/post/how-to-plot-your-data-on-maps-using-python-and-folium/

In [16]:
# Load plugins
from folium import plugins
import pandas as pd

# Download data for this example
stations_url = 'https://gbfs.divvybikes.com/gbfs/en/station_information.json'
stations = json.loads(requests.get(stations_url).text)['data']['stations']

# Create pandas object
stations = pd.json_normalize(stations)
# Create list of stations (latitude and longitude)
stations = stations[['lat', 'lon']]

# Create map
m = folium.Map([41.8781, -87.6298], zoom_start=11)

# convert to (n, 2) nd-array format for heatmap
stationArr = stations.values

# plot heatmap
m.add_child(plugins.HeatMap(stationArr, radius=15))

m
